In [13]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
:dep anyhow = { version = "1.0.75" }

In [14]:
:dep aoc2023 = { path = "../../" }

In [15]:
use aoc2023::utils::util;
use std::str::FromStr;
use anyhow::Error;

In [44]:
let lines: Vec<String> = util::lines_in("./input");
lines.len()

5

In [45]:
lines

["19, 13, 30 @ -2,  1, -2", "18, 19, 22 @ -1, -1, -2", "20, 25, 34 @ -2, -2, -4", "12, 31, 28 @ -1, -2, -1", "20, 19, 15 @  1, -5, -3"]

In [18]:
#[derive(Clone, PartialEq, Debug, Copy)]
pub struct Vec3d {
    pub p: f64,
    pub q: f64,
    pub r: f64,
}

impl FromStr for Vec3d {
    type Err = Error;

    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let mut splits = s.split(",");
        let err = || Error::msg("parse error");
        Ok(Vec3d::new(
            splits.next().ok_or(err())?.parse::<f64>()?,
            splits.next().ok_or(err())?.parse::<f64>()?,
            splits.next().ok_or(err())?.parse::<f64>()?,
        ))
    }
}

impl Vec3d {
    pub fn new(p: f64, q: f64, r: f64) -> Vec3d {
        Vec3d { p, q, r }
    }

    fn get(&self, i: usize) -> f64 {
        match i {
            0 => self.p,
            1 => self.q,
            2 => self.r,
            _ => panic!("unknown coord index"),
        }
    }
}

struct Hail {
    pos: Vec3d,
    vel: Vec3d,
}

impl Hail {
    fn m2d(&self) -> f64 {
        self.vel.q / self.vel.p
    }

    fn c2d(&self) -> f64 {
        self.pos.q - (self.m2d() * self.pos.p)
    }

    fn inter2d(&self, other: &Hail) -> (f64, f64) {
        let p = (self.c2d() - other.c2d()) / (other.m2d() - self.m2d());
        let q = (self.m2d() * p) + self.c2d();
        (p, q)
    }

    fn is_n_future(&self, a: f64, n: usize) -> bool {
        // Check if the 'velocity' and 'distance to a' has same sign
        (a - self.pos.get(n)) * self.vel.get(n) > 0.0
    }

    fn is_future(&self, p: f64, q: f64) -> bool {
        if self.vel.p != 0.0 {
            self.is_n_future(p, 0)
        } else {
            self.is_n_future(q, 1)
        }
    }
}

In [19]:
fn parse_vec_str(s: &str) -> Vec3d {
    let mut splits = s.split(", ");
    Vec3d::new(
        splits.next().unwrap().trim().parse::<f64>().unwrap(),
        splits.next().unwrap().trim().parse::<f64>().unwrap(),
        splits.next().unwrap().trim().parse::<f64>().unwrap(),
    )
}

In [46]:
let mut hails = vec![];
for line in lines {
    let (pos_str, vel_str) = line.split_once(" @ ").unwrap();
    hails.push(Hail {
        pos: parse_vec_str(pos_str),
        vel: parse_vec_str(vel_str),
    })
}

()

In [21]:
hails.len()

5

In [66]:
fn get_data(hails: &Vec<Hail>, count: i64, i: usize) -> Vec<(f64, f64)> {
    let vel_p = hails[i].vel.p as i64;
    let mut min_p = hails[i].pos.p as i64;
    let m = hails[i].m2d();
    let c = hails[i].c2d();
    let mut max_p = min_p + (vel_p * count);

    if vel_p < 0 {
        (min_p, max_p) = (max_p, min_p)
    }

    (min_p..max_p).map(|x| x as f64).map(|x| (x, (m * x) + c)).collect::<Vec<_>>()
}
let data0 = get_data(&hails, 20, 0);
let data1 = get_data(&hails, 20, 1);
let data2 = get_data(&hails, 20, 2);

In [70]:
extern crate plotters;
use plotters::prelude::*;

evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Hail Path", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(-25f64..40f64, -25f64..40f64)?;
    
    chart.configure_mesh()
        .draw()?;
    
    chart.draw_series(data0.iter().map(|(x,y)| Circle::new((*x,*y), 2, GREEN.filled())))?;
    chart.draw_series(data1.iter().map(|(x,y)| Circle::new((*x,*y), 2, BLUE.filled())))?;
    chart.draw_series(data2.iter().map(|(x,y)| Circle::new((*x,*y), 2, RED.filled())))?;
        
    Ok(())
}).style("width:60%")

Hail Path
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-20.0
 
 
 
-10.0
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
 
-20.0
 
 
 
-10.0
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0

In [74]:
(hails[0].pos.p, hails[0].vel.p)

(19.0, -2.0)

In [75]:
(hails[1].pos.p, hails[1].vel.p)

(18.0, -1.0)

xrt1 = 19 - 2t1
xrt2 = 18 - t2


vrx = (xrt2 - xrt1) / (t2 - t1)
    = 